In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP

/content/gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP


In [4]:
!ls

layer1.json	     preprossData.csv  RecipeGenerator
LICENSE		     README.md	       RecipeGenerator.xcodeproj
Preprocessing.ipynb  Recipe	       RecipeTraining.ipynb


In [5]:
import re
import pandas as pd

In [6]:
import json
with open('layer1.json') as json_file:
    data = json.load(json_file)

In [7]:
recipes = data
recipes[0]['title']

'Worlds Best Mac and Cheese'

In [8]:
def get_ingredients(recipe):    
    ingredients = recipe['ingredients']
    measurements = ['oz', 'ounces','ounce', 'cups', 'cup', 'teaspoon', '/', 'to', 'lbs','lb', 'tsp', 'jar']
    item_list = []
    for item in ingredients:
        item = item['text']
        item = item.lower()
    #     item = item.split('(')[0].strip() # remove items in brackets
        item = re.sub(r'\s*\([^)]*\)', '', item)
        item = ''.join([char for char in item if not char.isdigit()]) # remove numbers
        for measurement in measurements:
            item = item.replace(measurement, '').strip() # remove measurements
        item.strip()
        if item:
            item_list.append(item)

    ingredient_list = '|'.join(item_list)
    return ingredient_list

In [12]:
def get_instructions(recipe):
    instructions = recipe['instructions']
    instr_str = ""
    for instr in instructions:
        instr_str += "recipe:" +instr['text'].lower()
    return instr_str

get_instructions(recipes[32])

'recipe:for the cupcakes: preheat the oven to 350 degrees f. fill the cupcake pan with paper liners.recipe:beat the granulated sugar and butter until light and fluffy in a standing mixer.recipe:beat in the eggs.recipe:add the lemon zest and juice and beat until combined.recipe:sift the flour, baking powder, baking soda and salt in a separate bowl.recipe:alternately add the buttermilk and flour mixture to the standing mixer, beating well between each addition.recipe:fill the cupcake liners two-thirds full.recipe:bake the cupcakes until a toothpick inserted comes out clean, 18 to 22 minutes.recipe:refrigerate the cupcakes until they are completely cooled.recipe:for the filling: cook 1/3 cup water, 1 cup of the blueberries and granulated sugar over medium heat for 10 minutes, stirring frequently.recipe:once the blueberries begin to burst, add the remaining 1 cup blueberries and cook on medium heat, 8 to 10 minutes.recipe:the filling is done when it coats the back of a metal spoon.recipe:f

In [15]:
in_recipe = []

for recipe in recipes:
    ingredients = get_ingredients(recipe)
    instr = get_instructions(recipe)
    qna = { "title": recipe['title'],
            "text": ingredients+instr}
    in_recipe.append(qna)

In [16]:
df = pd.DataFrame(in_recipe)

In [17]:
df.head()

,title,text
0,Worlds Best Mac and Cheese,"penne|beechers flagship cheese sauce|cheddar, ..."
1,Dilly Macaroni Salad Recipe,c. eow macaroni|c. cubed american cheese|c. sl...
2,Gazpacho,"maes, quartered|kosher salt|red onion, cut in ..."
3,Crunchy Onion Potato Bake,"milk|water|butter|mashed potaes, box, homesty..."
4,Cool 'n Easy Creamy Watermelon Pie,package watermelon gelatin|boiling water|packa...


In [22]:
len(df.text.unique()), len(df)

(1023257, 1029720)

In [19]:
df.to_csv('preprossData.csv')